In [11]:
import os, sys

try:
    import boto3
    print('Boto3 imported successfully')
except Exception as e:
    print(e)
    sys.exit(1)

Boto3 imported successfully


In [21]:
source_region='us-east-1'
dest_region='us-east2'

session=boto3.session.Session(profile_name='root')
ec2_source_client=session.client(service_name='ec2', region_name=source_region)

sts_client=session.client(service_name='sts', region_name='us-east-1')
account_id=sts_client.get_caller_identity().get('Account')

f1={'Name':'tag:backup', 'Values':['yes']}

backup_snap=[]
for each_snap in ec2_source_client.describe_snapshots(OwnerIds=[account_id], Filters=[f1]).get('Snapshots'):
    backup_snap.append(each_snap.get('SnapshotId'))
    
ec2_dest_client=session.client(service_name='ec2',region_name=dest_region)


for each_source_snapid in backup_snap:
    print('Taking backup of {}into the region {}'.format(each_source_snapid,dest_region))
    ec2_dest_client.cpoy_snapshot(
        Description='Disaster recovery',
        SourceRegion=source_region,
        SourceSnapshotId=each_source_snapid,
        )
    
    
print('EBS snapshot copy to destion region is completed')
print('modifyng tags for which backup is done')

for each_source_snapid in backup_snap:
    
    print('deleting old tags for {}'.format(each_source_snapid))
    ec2_source_client.delete_tags(
        Resources=[ach_source_snapid],
        Tags=[
        {
        'Key':'backup',
        'Value':'yes'    
        }    
        ]
        )
    print("creating new tags for {}".format(each_source_snapid))   
    ec2_source_client.create_tags(   
        Resources=[ach_source_snapid],
        Tags=[
        {
        'Key':'backup',
        'Value':'completed'    
        }    
        ]
        )
    
    
    
    

EBS snapshot copy to destion region is completed
modifyng tags for which backup is done
